In [1]:
import pandas as pd
import numpy as np


In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
sub=pd.read_csv("sample_submission.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               165034 non-null  int64  
 1   CustomerId       165034 non-null  int64  
 2   Surname          165034 non-null  object 
 3   CreditScore      165034 non-null  int64  
 4   Geography        165034 non-null  object 
 5   Gender           165034 non-null  object 
 6   Age              165034 non-null  float64
 7   Tenure           165034 non-null  int64  
 8   Balance          165034 non-null  float64
 9   NumOfProducts    165034 non-null  int64  
 10  HasCrCard        165034 non-null  float64
 11  IsActiveMember   165034 non-null  float64
 12  EstimatedSalary  165034 non-null  float64
 13  Exited           165034 non-null  int64  
dtypes: float64(5), int64(6), object(3)
memory usage: 17.6+ MB


In [24]:
cols=train.columns

In [5]:
train['Geography'].value_counts()

Geography
France     94215
Spain      36213
Germany    34606
Name: count, dtype: int64

In [5]:
dtypes=[]
for i in cols:
    dtypes.append(train[i].dtype)
    

In [6]:
schema={"id":"int64","CustomerId":"object"}

In [17]:
cols=["id","CustomerId"]

In [29]:
schema.values()

dict_values(['int64', 'object'])

In [32]:
for i,j in schema.items():
    if i in train.columns and train[i].dtype==j :
        print(True)
    else :
        print(False)
    

True
False


In [9]:
dtypes

[dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64')]

In [25]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import xgboost
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler,OneHotEncoder

In [26]:
X=train.drop(columns="Exited")
y=train['Exited']

In [28]:
def transform(df):
    cat_cols=["Geography","Gender"]
    drop_cols=['id',"CustomerId","Surname"]
    num_cols=['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary']
    df2=df.drop(columns=drop_cols)
    num_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='median')),
                ('scaler',StandardScaler())
                ])
    cat_pipeline=Pipeline(
                steps=[
                ('imputer',SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',OneHotEncoder(sparse=False,handle_unknown="ignore"))
                ]
            )
    preprocessor_cat=ColumnTransformer([
            ('cat_pipeline',cat_pipeline,cat_cols)
            ])
    preprocessor_num=ColumnTransformer([
            ('num_pipeline',num_pipeline,num_cols)
            ])
    result=preprocessor_cat.fit_transform(df2)
    columns=preprocessor_cat.named_transformers_['cat_pipeline'].named_steps['ordinalencoder'].get_feature_names_out(cat_cols)
    cat=pd.DataFrame(result,columns=columns)
    result2=preprocessor_num.fit_transform(df2)
    num=pd.DataFrame(result2,columns=df2[num_cols].columns)
    return pd.concat([num,cat],axis=1)


In [29]:
final_df=transform(X)

E:\SOFTWARES\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [30]:
X_train,X_test,y_train,y_test=train_test_split(final_df,y,test_size=0.2,random_state=1)

In [31]:
smote=SMOTE(random_state=1)
x_smote,y_smote=smote.fit_resample(X_train,y_train)
print(y_smote.value_counts())
print(y_train.value_counts())

0    104119
1    104119
Name: Exited, dtype: int64
0    104119
1     27908
Name: Exited, dtype: int64


In [18]:
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix,classification_report
from xgboost import XGBClassifier

In [15]:
xgb_classifier = XGBClassifier(subsample= 0.9, n_estimators= 200, 
                               max_depth= 5,learning_rate=0.1,colsample_bytree=0.9)

In [16]:
xgb_classifier.fit(x_smote,y_smote)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [19]:
preds=xgb_classifier.predict(X_test)

In [20]:
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91     25994
           1       0.66      0.65      0.65      7013

    accuracy                           0.85     33007
   macro avg       0.78      0.78      0.78     33007
weighted avg       0.85      0.85      0.85     33007



In [22]:
from mlxtend.feature_selection import SequentialFeatureSelector as sf
selected_features=sf(xgb_classifier,k_features=8,forward=False)

In [ ]:
cols=selected_features.fit(x_smote,y_smote)